In [2]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

In [3]:
sc = SparkContext().getOrCreate()
spark = SparkSession(sc)

22/05/07 03:17:14 WARN Utils: Your hostname, Junuui-MacBookAir.local resolves to a loopback address: 127.0.0.1; using 172.30.1.51 instead (on interface en0)
22/05/07 03:17:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/07 03:17:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [190]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification

train, test = make_classification(n_features=5, n_classes=2, n_samples=100, n_informative=3, random_state=42)
df = pd.DataFrame(train)
df['label'] = test
print(df.shape)
df.head()

(100, 6)


,0,1,2,3,4,label
0,-0.516603,0.146493,0.752866,1.603592,0.743463,1
1,-0.319406,-1.106124,2.780528,2.395492,-0.850553,1
2,-0.710865,-0.109481,1.966707,1.732722,-0.219927,1
3,0.652532,-0.797940,0.177784,-1.161287,-1.453970,0
4,0.760074,-0.517866,-0.480863,-2.111619,-1.539946,0


In [191]:
pdf = spark.createDataFrame(df)
pdf.show(3)

+--------------------+--------------------+------------------+------------------+-------------------+-----+
|                   0|                   1|                 2|                 3|                  4|label|
+--------------------+--------------------+------------------+------------------+-------------------+-----+
| -0.5166027256607268|  0.1464926805027933|0.7528659486995097|1.6035924552628822| 0.7434633835995562|    1|
|-0.31940571768117404|  -1.106123551531058| 2.780527653332907|2.3954923577012455|-0.8505528241369291|    1|
| -0.7108649696722994|-0.10948051769929945|1.9667072767000313| 1.732722411292773|-0.2199273564146513|    1|
+--------------------+--------------------+------------------+------------------+-------------------+-----+
only showing top 3 rows



In [192]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler

va = VectorAssembler(inputCols=[str(i) for i in range(5)], outputCol='features')
pipeline = Pipeline(stages=[va])
piped_data = pipeline.fit(pdf).transform(pdf)

In [193]:
train, test = piped_data.randomSplit([0.8, 0.2])

In [194]:
# Import LogisticRegression
from pyspark.ml.classification import LogisticRegression

# Create a LogisticRegression Estimator
lr = LogisticRegression().setLabelCol('label').setFeaturesCol('features')

In [195]:
# Import the evaluation submodule
import pyspark.ml.evaluation as evals

# Create a BinaryClassificationEvaluator
evaluator = evals.BinaryClassificationEvaluator(metricName='areaUnderROC')

In [196]:
# Import the tuning submodule
import pyspark.ml.tuning as tune

# Create the parameter grid
grid = tune.ParamGridBuilder()

# Add the hyperparameter
# grid = grid.addGrid(lr.regParam, [0, .1])
grid = grid.addGrid(lr.elasticNetParam, [0, 1])

# Build the grid
grid = grid.build()

In [197]:
print(len(grid))
print(grid[0])

2
{Param(parent='LogisticRegression_21b026f13963', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0}


In [198]:
# Create the CrossValidator
cv = tune.CrossValidator(estimator=lr,
               estimatorParamMaps=grid,
               evaluator=evaluator
               )

In [199]:
# Fit cross validation models
models = cv.fit(train)

# Extract the best model
best_lr = models.bestModel

In [200]:
best_lr.getElasticNetParam()

0.0

In [201]:
test_results = best_lr.transform(test)
print(evaluator.evaluate(test_results))

0.888888888888889


In [203]:
test_results.head(5)

[Row(0=-2.181158974887847, 1=2.7153870623466627, 2=0.9046546571743399, 3=-0.23442493593144698, 4=0.5455859017076543, label=1, features=DenseVector([-2.1812, 2.7154, 0.9047, -0.2344, 0.5456]), rawPrediction=DenseVector([-4.148, 4.148]), probability=DenseVector([0.0156, 0.9844]), prediction=1.0),
 Row(0=-1.0991924198831942, 1=2.20378063202382, 2=-2.4402521672927477, 3=2.1657985801707715, 4=4.7009085537637745, label=1, features=DenseVector([-1.0992, 2.2038, -2.4403, 2.1658, 4.7009]), rawPrediction=DenseVector([-4.4648, 4.4648]), probability=DenseVector([0.0114, 0.9886]), prediction=1.0),
 Row(0=-0.7108649696722994, 1=-0.10948051769929945, 2=1.9667072767000313, 3=1.732722411292773, 4=-0.2199273564146513, label=1, features=DenseVector([-0.7109, -0.1095, 1.9667, 1.7327, -0.2199]), rawPrediction=DenseVector([-7.183, 7.183]), probability=DenseVector([0.0008, 0.9992]), prediction=1.0),
 Row(0=-0.6417888414630424, 1=0.6730550962253387, 2=0.019259293225993512, 3=1.3295287847414041, 4=1.3681998314